In [1]:
# !pip install pennylane nftopt

In [2]:
# import numpy as np
# arr = []
# for i in range(256):
#   arr.append(1/256)

# arr = np.array(arr)
# print(arr)

In [3]:
import numpy as np
arr = np.random.uniform(low=-1, high=1,size=(8))
# arr = np.array([[0.1,-0.1,-0.1,0.1,-0.1,0.1,-0.2,0.2]])
print(arr)

[ 0.37355263 -0.44361671 -0.39660804  0.04016908 -0.54340399  0.24815682
 -0.31465694 -0.99103463]


In [4]:
np.unique(arr)

array([-0.99103463, -0.54340399, -0.44361671, -0.39660804, -0.31465694,
        0.04016908,  0.24815682,  0.37355263])

In [5]:
# Rearranging the array so that there is no index confusion
arr1 = []
for i in range(len(arr)):
  if(arr[i] > 0):
    arr1.append(arr[i])

for i in range(len(arr)):
  if(arr[i] < 0):
    arr1.append(arr[i])

arr1 = np.array([arr1])
print(arr1)

[[ 0.37355263  0.04016908  0.24815682 -0.44361671 -0.39660804 -0.54340399
  -0.31465694 -0.99103463]]


In [6]:
summer = np.sum(np.abs(arr))
print(summer)
normalized_X = arr/summer
print(normalized_X)

3.3511988514498015
[ 0.11146836 -0.13237553 -0.11834811  0.01198648 -0.16215212  0.07405016
 -0.09389384 -0.2957254 ]


In [7]:
#defining parameters
n_qubits = 4 #( 3 + 1 ancillary)
depth = 4

In [8]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod

In [9]:
dev = qml.device("default.qubit",wires=n_qubits)
dev

<default.qubit device (wires=4) at 0x793eeb977dc0>

In [10]:
def ansatz_layer(layer_weights, depth, n_qubits):
    for dep in range(depth):
      for wire in range(0,n_qubits):
        qml.RY(layer_weights[wire+(n_qubits*dep)], wires=wire)

      qml.CZ(wires=[0,1])
      qml.CZ(wires=[1,2])
      qml.CZ(wires=[2,3])
      qml.CZ(wires=[3,0])

    qml.Hadamard(0)

In [11]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [12]:
@qml.qnode(dev,diff_method="backprop")
def circuit(weights):
  # since depth 4 was used
  ansatz_layer(weights,depth=depth, n_qubits=n_qubits)
  # qml.state() applies Ua to Ini State, which gives Appro_a
  return qml.state()

In [13]:
def variational_classifier(weights, x):
    # weights are thetas
    return np.real(circuit(weights))

In [14]:
# temp = []
# for i in range(len(normalized_X[0])):
#   temp.append(np.sqrt(normalized_X[0][i]))
# x = np.array(temp)
# print(x)

temp = []
for i in range(len(normalized_X)):
  val = np.sqrt(np.abs(normalized_X[i]))
  if (normalized_X[i] < 0):
    val = val * -1
  temp.append(val)

x = np.array(temp)
print(x)

# # [ 0.1 -0.1 -0.1  0.1 -0.1  0.1 -0.2  0.2]
# x = np.array([0.31622776601683794,-0.31622776601683794,-0.31622776601683794,0.31622776601683794,-0.31622776601683794,0.31622776601683794,-0.4472135954999579,0.4472135954999579])
# print(x)

[ 0.33386877 -0.36383448 -0.3440176   0.10948279 -0.40268116  0.2721216
 -0.30642102 -0.5438064 ]


In [15]:
sum = 0
for i in range(len(x)):
  sum += x[i]**2
print(sum)

1.0


In [16]:
# def F1_loss(a,Appro_a):
#   #   Define the cost function
#   F1=(a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2))
#   F2=(a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2))
#   F3=(a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2))

#   #   Prepare the Bell state
#   Bell_State=np.ones((8,1))/np.linalg.norm(np.ones((8,1)))
#   #   The sum of vector a
#   Sum_a=np.sum(a)
#   sqrt_D = np.linalg.norm(np.ones((8,1)))
#   UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F1+F2+F3)
#   return np.real(UaFunction)

In [17]:
# D = 2**d
# D+ = k
# D- = l
def F1_loss(a,Appro_a):
  #   Define the cost function
  F2 = 0.
  k = []
  k_ind = []
  l = []
  l_ind = []
  # Split a into k and l, store indices as well(+ve and -ve)
  for i in range(len(a)):
    if(a[i] >= 0):
      k.append(a[i])
      k_ind.append(i)
    else:
      l.append(a[i])
      l_ind.append(i)

  for i in range(len(k)):
    F2 += (k[i]**2)*np.log2((Appro_a[k_ind[i]]**2)/(k[i]**2))
  D = 2 ** (n_qubits-1)
  for i in range(len(l)):
    F2 += (l[i]**2)*np.log2((Appro_a[l_ind[i]+D]**2)/(l[i]**2))


  #   Prepare the Bell state
  states_with_ancillary = 2**(n_qubits)
  Bell_State=np.ones((states_with_ancillary,1))/np.linalg.norm(np.ones((states_with_ancillary,1)))
  #   The sum of vector a
  Sum_a=np.sum(np.abs(a))
  sqrt_D = np.sqrt(2) * np.linalg.norm(np.ones((D,1)))
  UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F2)
  return np.real(UaFunction)

In [18]:
a= x

In [19]:
print(a)

[ 0.33386877 -0.36383448 -0.3440176   0.10948279 -0.40268116  0.2721216
 -0.30642102 -0.5438064 ]


In [20]:
def cost(weights, X=x):
    # X is our preparation state
    Appro_a = variational_classifier(weights, X)
    return F1_loss(X, Appro_a)

In [21]:
weights = 4*np.pi*np.random.rand(n_qubits * depth)
result= optimize.minimize(cost,weights, method=nftmethod, options={'maxfev':2000})


In [22]:
result.x

array([ 5.71650351,  6.31368545,  2.22212513, 13.12137346, 10.38234483,
        3.08274821,  4.35672108,  1.00910184,  4.6029608 ,  5.03021641,
       11.20934093, -0.61577448,  6.23123078,  0.11830317,  7.22266903,
        9.86314686])

In [23]:
print(a)

[ 0.33386877 -0.36383448 -0.3440176   0.10948279 -0.40268116  0.2721216
 -0.30642102 -0.5438064 ]


In [24]:
k_ind = []
l_ind = []
for i in range(len(a)):
    if(a[i] >= 0):
      k_ind.append(i)
    else:
      l_ind.append(i)
print(k_ind)
print(l_ind)

[0, 3, 5]
[1, 2, 4, 6, 7]


In [25]:
# Left most value in state vector is the 0th qubit which the top one in circuit - Pennylane
predictions=variational_classifier(result.x,a)
print(predictions)

modified_predictions = []
D = 2 ** (n_qubits - 1)
for i in range(len(a)):
  if(i in k_ind):
    # positive we take from |0> of zeroth qubit
    modified_predictions.append(predictions[i])
  else:
    modified_predictions.append(-1 * predictions[i + D])

modified_predictions = np.array(modified_predictions)
print(modified_predictions)

acc = accuracy(x, modified_predictions)
print(modified_predictions.numpy(), x.numpy())
print(acc)

[-0.03772637  0.13735294  0.07027094  0.06858903  0.06663741  0.08222756
  0.06099133  0.14308302  0.02510106  0.35511322  0.36837734  0.18295927
  0.46192534  0.15196547  0.25695381  0.57988673]
[-0.03772637 -0.35511322 -0.36837734  0.06858903 -0.46192534  0.08222756
 -0.25695381 -0.57988673]
[-0.03772637 -0.35511322 -0.36837734  0.06858903 -0.46192534  0.08222756
 -0.25695381 -0.57988673] [ 0.33386877 -0.36383448 -0.3440176   0.10948279 -0.40268116  0.2721216
 -0.30642102 -0.5438064 ]
0.7281398067583531
